# End-To-End Walkthrough

> How to build a website in FastHTML step-by-step
- order: 1
- image: ../images/card.png
- aliases: [/tutorial.html]

In [ ]:
#|hide
from nbdev.showdoc import show_doc
from nbdev.qmd import div

The written tutorial below shows you how to create a Python package from scratch using nbdev.

Alternatively, you can watch this video tutorial where Jeremy Howard and Hamel Husain guide you through a similar process step by step:

::: {.text-center}

<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/l7zS8Ld4_iA" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen style="max-width: 100%; margin: auto" class="rounded"></iframe>

:::

## Installation

You'll need the following software to complete the tutorial, read on for specific installation instructions:

1. Python
2. A Python package manager: we recommend conda or pip
3. FastHTML
4. Web browswer
5. Railway.app account

If you haven't worked with Python before, we recommend getting started with the [Anaconda Individual Edition](https://www.anaconda.com/products/individual) and using the conda package manager.

Note that you will only need to follow the steps in the installation section once per environment. If you create a new repo, you won't need to redo these.

### Install FastHTML

The next step is to install nbdev itself. JupyterLab comes with its own terminal, so we'll use that moving forward.

In the Launcher, scroll down to the "Other" section, then click "Terminal". If the Launcher isn't opened, you can open it by clicking "File" → "New Launcher".

A new tab should open with a blank terminal -- it might not look exactly the same, depending on how your shell is configured:

![](images/jupyter-blank-terminal.png){fig-align="center"}

For Mac, Windows and Linux, enter:

```sh
pip install python-fasthtml
```

## First steps

By the end of this section you'll have your own FastHTML website with tests deployed to railway.app.

### Create a hello world

Get a fresh folder to put all of your files for your new project in and in it place a single `main.py` with the following code:

```python
from fasthtml.common import *

app = FastHTML()
rt = app.route

@rt('/')
def get():
    return 'Hello, world!'

run_uv()
```

Finally, run `python main.py` in your terminal and open your browser to the linked printed.

In addition to viewing the site in a browswer, you can use a test client from starlette, which we will be taking advantage of for our tests. Open a new file named `test.py` and place the following code in it:

```python
from starlette.testclient import TestClient

client = TestClient(app)
r = client.get('/')
print(r)
```

### QuickDraw: A FastHTML Adventure 🎨✨

The end website of this tutorial will be QuickDraw, a real-time collaborative drawing app using FastHTML. Here is what the final site will look like:

(IMAGE)

#### Drawing Rooms

Draw rooms are the core concept of our application. Each room represents a separate drawing space where multiple users can collaborate in real-time. Here's a detailed breakdown:

1. Room Creation and Storage

```python
db = database('data/drawapp.db')
rooms = db.t.rooms
if rooms not in db.t:
    rooms.create(id=int, name=str, created_at=str, pk='id')
Room = rooms.dataclass()
```

* We're using a SQLite database (via FastLite) to store our rooms.
* Each room has an id (integer), a name (string), and a created_at timestamp (string).
* The Room dataclass is automatically generated based on this structure.

2. Creating a room

```python
@rt("/")
def get():
    create_room = Form(Input(id="room-name", name="name", placeholder="New Room Name"),
                       Button("Create Room"),
                       hx_post="/rooms", hx_target="#rooms-list", hx_swap="afterbegin")
    rooms_list = Ul(*rooms(order_by='id DESC'), id='rooms-list')
    return Titled("DrawCollab", 
                  H1("DrawCollab"),
                  create_room, rooms_list)

@rt("/rooms")
async def post(room:Room):
    room.created_at = datetime.now().isoformat()
    new_room = rooms.insert(room)
    return Li(A(new_room.name, href=f"/rooms/{new_room.id}"))
```

* When a user submits the "Create Room" form, this route is called.
* It creates a new Room object, sets the creation time, and inserts it into the database.
* It returns an HTML list item with a link to the new room, which is dynamically added to the room list on the homepage thanks to HTMX.

3. Let's give our rooms shape

```python
@rt("/rooms/{id}")
async def get(id:int):
    room = rooms[id]
    return Titled(f"Room: {room.name}", H1(f"Welcome to {room.name}"), P("Drawing canvas coming soon!"))
```

* This route renders the interface for a specific room.
* It fetches the room from the database and renders a title, heading, and paragraph.


Here is the full code so far:
```python
from fasthtml.common import *
from datetime import datetime

db = database('data/drawapp.db')
rooms = db.t.rooms
if rooms not in db.t:
    rooms.create(id=int, name=str, created_at=str, pk='id')
Room = rooms.dataclass()

app = FastHTML(hdrs=(picolink,))
rt = app.route

@rt("/")
def get():
    create_room = Form(Input(id="room-name", name="name", placeholder="New Room Name"),
                       Button("Create Room"),
                       hx_post="/rooms", hx_target="#rooms-list", hx_swap="afterbegin")
    rooms_list = Ul(*rooms(order_by='id DESC'), id='rooms-list')
    return Titled("DrawCollab", 
                  H1("DrawCollab"),
                  create_room, rooms_list)

@rt("/rooms")
async def post(room:Room):
    room.created_at = datetime.now().isoformat()
    new_room = rooms.insert(room)
    return Li(A(new_room.name, href=f"/rooms/{new_room.id}"))

@rt("/rooms/{id}")
async def get(id:int):
    room = rooms[id]
    return Titled(f"Room: {room.name}", H1(f"Welcome to {room.name}"), P("Drawing canvas coming soon!"))

if __name__ == "__main__": run_uv()
```

Now run `python main.py` in your terminal and open your browser to the linked printed. You should see a page with a form to create a new room and a list of existing rooms.

#### The Canvas - Let's Get Drawing! 🖌️

Time to add the actual drawing functionality. We'll use Fabric.js for this:
```python
# ... (keep the previous imports and database setup)

@rt("/rooms/{id}")
async def get(id:int):
    room = rooms[id]
    canvas = Div(id="canvas")
    color_picker = Input(type="color", id="color-picker", value="#000000")
    brush_size = Input(type="range", id="brush-size", min="1", max="50", value="10")
    
    js = """
    var canvas = new fabric.Canvas('canvas');
    canvas.isDrawingMode = true;
    canvas.freeDrawingBrush.color = '#000000';
    canvas.freeDrawingBrush.width = 10;
    
    document.getElementById('color-picker').onchange = function() {
        canvas.freeDrawingBrush.color = this.value;
    };
    
    document.getElementById('brush-size').oninput = function() {
        canvas.freeDrawingBrush.width = parseInt(this.value, 10);
    };
    """
    
    return Titled(f"Room: {room.name}",
                  canvas,
                  Div(color_picker, brush_size),
                  Script(src="https://cdnjs.cloudflare.com/ajax/libs/fabric.js/5.3.1/fabric.min.js"),
                  Script(js))

# ... (keep the run_uv() part)
```

Now we've got a drawing canvas! FastHTML makes it easy to include external libraries and add custom JavaScript.

### Deploying to Railway

You can deploy your website to a number of hosting providers, for this tutorial we'll be using Railway.

1. Create a Railway account
2. Install the Railway CLI
3. Deploy your website

To make deploying your website as easy as possible, FastHTMl comes with a built in CLI tool that will handle most of the deployment process for you.

To deploy your website, run the following command in your terminal:

```sh
fh_railway_deploy
```

### Conclusion: You're a FastHTML Artist Now! 🎨🚀

Congratulations! You've just built a sleek, interactive web application using FastHTML. Let's recap what we've learned:

FastHTML allows you to create dynamic web apps with minimal code.
HTMX integration enables seamless, partial page updates without full reloads.
WebSockets power real-time communication between clients and the server.
FastHTML plays well with external libraries like Fabric.js, allowing you to create rich, interactive experiences.
Building complex UI components and real-time features is a breeze with FastHTML.

You've taken your first steps into the world of FastHTML development. From here, the possibilities are endless! Maybe add the ability to save drawings, implement different drawing tools, or even add a gallery of completed masterpieces. Whatever you choose to build next, FastHTML has got your back.
Now go forth and create something awesome! And remember, in the world of web development, as in art, there are no bugs - only features waiting to be discovered. Happy coding! 🖼️🚀